# Benchmark comparing MongoDB sync vs. async

In [1]:
import time
import asyncio
import contextlib
from pymongo import MongoClient
from motor.motor_asyncio import AsyncIOMotorClient

@contextlib.contextmanager
def timer(description):
    start_time = time.time()
    yield
    end_time = time.time()
    print('{} took {:5.4f} seconds'.format(description, end_time - start_time))
    
sync_client = MongoClient().aiotest.testcol
aio_client = AsyncIOMotorClient(max_pool_size=10).aiotest.testcol
loop = asyncio.get_event_loop()

In [2]:
N = 1000

In [3]:
_ = sync_client.insert({'abc': i} for i in range(N))

## Synchronous Requests

In [4]:
with timer('sequential sync read'):
    for i in range(N):
        sync_client.find_one({'abc': i})

sequential sync read took 0.3548 seconds


## Asynchronous Requests

In [5]:
with timer('sequential async read'):
    for i in range(N):
        loop.run_until_complete(aio_client.find_one({'abc': i}))

sequential async read took 0.5510 seconds


Using an async for-loop is not much faster:

In [6]:
async def f():
    async for doc in aio_client.find():
        pass
    
with timer('sequential async read with cursor'):
    loop.run_until_complete(f())

sequential async read with cursor took 0.0143 seconds


In [7]:
for chunk_size in range(1, 20):
    with timer('parallel async read with {} parallel requests'.format(chunk_size)):
        for i in range(N // chunk_size):
            # interesting part:
            coros = [aio_client.find_one({'abc': (i * chunk_size) + j})
                     for j in range(chunk_size)]
            loop.run_until_complete(asyncio.gather(*coros))

parallel async read with 1 parallel requests took 0.5825 seconds
parallel async read with 2 parallel requests took 0.3421 seconds
parallel async read with 3 parallel requests took 0.2892 seconds
parallel async read with 4 parallel requests took 0.2624 seconds
parallel async read with 5 parallel requests took 0.2528 seconds
parallel async read with 6 parallel requests took 0.2457 seconds
parallel async read with 7 parallel requests took 0.2376 seconds
parallel async read with 8 parallel requests took 0.2409 seconds
parallel async read with 9 parallel requests took 0.2287 seconds
parallel async read with 10 parallel requests took 0.2477 seconds
parallel async read with 11 parallel requests took 0.2754 seconds
parallel async read with 12 parallel requests took 0.2692 seconds
parallel async read with 13 parallel requests took 0.2364 seconds
parallel async read with 14 parallel requests took 0.2307 seconds
parallel async read with 15 parallel requests took 0.2309 seconds
parallel async read

In [8]:
sync_client.drop()